In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import requests, time, json, pytz, asyncio, random
import pandas as pd
import SpaceTraders as ST
from SpaceTraders import io, F_utils, F_nav, F_trade, scripts, fleet_resource_manager
from SpaceTraders.controllers import system_market_intel as MI
from datetime import datetime

In [60]:
ST.get_request('/my/agent').json()

{'data': {'accountId': 'cmhhh9gk1002vtm16u5rl9pf0',
  'symbol': 'RYVIOS',
  'headquarters': 'X1-QT34-A1',
  'credits': 1196494,
  'startingFaction': 'COSMIC',
  'shipCount': 3}}

In [4]:
ST.get_request('/agents/HIVE').json()

{'data': {'symbol': 'HIVE',
  'headquarters': 'X1-QT34-A1',
  'credits': 21683938,
  'startingFaction': 'COSMIC',
  'shipCount': 17}}

In [5]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-D41', 'BURN', verbose=True)

[INFO] Ship RYVIOS-1 refueled 0 units. Total cost: 0 cr (72 cr/u).
[ERROR] Ship RYVIOS-1 is already at destination X1-TP30-D41. Navigation cancelled.


False

In [9]:
F_nav.get_ship_nav('RYVIOS-1')

{'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-D41',
 'route': {'destination': {'symbol': 'X1-TP30-D41',
   'type': 'PLANET',
   'systemSymbol': 'X1-TP30',
   'x': 41,
   'y': -75},
  'origin': {'symbol': 'X1-TP30-H52',
   'type': 'MOON',
   'systemSymbol': 'X1-TP30',
   'x': 40,
   'y': -19},
  'departureTime': '2025-11-04T17:42:20.961Z',
  'arrival': '2025-11-04T17:42:55.961Z'},
 'status': 'DOCKED',
 'flightMode': 'BURN'}

In [33]:
def print_nav_info(ship):
    nav = F_nav.get_ship_nav(ship)
    if nav['status'] == 'IN_TRANSIT':
        arrival_time = nav['arrival']
        print(ship, "is navigating to", nav['waypointSymbol'], "- ETA", nav['arrival'], f"({F_nav.get_transit_time_seconds(ship):.0f} seconds remaining).")
    else:
        print(f"{ship} is {nav['status']} at {nav['waypointSymbol']}.")
        inv = F_trade.get_ship_cargo(ship)['inventory']
        if len(inv):
            print("[CARGO]")
            for i in inv:
                print(f"  - {i['units']} {i['symbol']}")

In [35]:
print_nav_info('RYVIOS-1')

RYVIOS-1 is DOCKED at X1-TP30-A1.


In [36]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-A2', 'BURN')

[INFO] Ship RYVIOS-1 refueled 2 units. Total cost: 144 cr (72 cr/u).
[INFO] Ship RYVIOS-1 en-route to X1-TP30-A2. Arrival at 2025-11-04T20:20:51.918 (15.0 seconds).


True

In [102]:
datetime.fromisoformat('2025-11-04T22:32:42.682+00:00')

datetime.datetime(2025, 11, 4, 22, 32, 42, 682000, tzinfo=datetime.timezone.utc)

In [113]:
F_utils.ts_delta_seconds('2025-11-04T22:32:42.682Z') // 60

-12.0

In [161]:
F_nav.get_transit_time_seconds('RYVIOS-2')

2.909733

In [164]:
constrsite = "X1-TP30-I55"
ST.get_request(f'/systems/X1-TP30/waypoints/{constrsite}/construction').json()

{'data': {'symbol': 'X1-TP30-I55',
  'materials': [{'tradeSymbol': 'FAB_MATS', 'required': 1600, 'fulfilled': 0},
   {'tradeSymbol': 'ADVANCED_CIRCUITRY', 'required': 400, 'fulfilled': 0},
   {'tradeSymbol': 'QUANTUM_STABILIZERS', 'required': 1, 'fulfilled': 1}],
  'isComplete': False}}

In [ ]:
def make_async(f):
    """ Wrapper that hands blocking function f to a separate thread, effectively making it asynchronous. """
    async def asyncified(*args, **kwargs):
        return await asyncio.to_thread(f, *args, **kwargs)
    return asyncified

@make_async
def blocking1():
    print("Blocking 1 - 4s")
    time.sleep(4)

@make_async
def blocking2():
    print("Blocking 2 - 1s")
    time.sleep(1)

def blocking3():
    print("  Blocking 3")
    time.sleep(0.1)

def blocking4():
    print("  Blocking 4")
    time.sleep(0.1)

@make_async
def blocking5():
    print("Blocking 5 - 2s")
    blocking3()
    time.sleep(1.8)
    blocking4()

async def main():
    for _ in range(5):
        await asyncio.gather(blocking1(), blocking2())


In [31]:
F_nav.get_transit_time_seconds('RYVIOS-1')

-716.976149

In [21]:
F_trade.get_total_profit_from_trade('RYVIOS-1', 'X1-TP30-D41', 'X1-TP30-H52', '2025-11-07T09:27:00Z')

49365

In [151]:
fleet_resource_manager.release_ship("RYVIOS-1")

True

In [129]:
fleet_resource_manager.set_ship_blocked_status("RYVIOS-1", True)

True

In [153]:
fleet_resource_manager.lock_ship("RYVIOS-1", "MANUAL", 1)

True

In [132]:
fleet_resource_manager.get_ship_blocked_status("RYVIOS-1")

False

In [137]:
fleet_resource_manager.get_ship_controller("RYVIOS-1")

(None, -1)

In [243]:
fleet_resource_manager.request_ship('RYVIOS-2', 'USER', 0)

[WARNING] Fleet resources has detected a moving ship without controller: RYVIOS-2.


False

In [4]:
# Simulate work function
async def update_market(ship, market):
    fleet_resource_manager.set_ship_blocked_status(ship, True)
    print(f"{ship} is en-route to {market}.")
    if random.random() < 0.65:
        # Faster task
        await asyncio.sleep(0.1 + (random.randint(10, 150) / 100.0))
    else:
        # Slower task
        await asyncio.sleep(1 + (random.randint(300, 700) / 100.0))
    success = random.random() < 0.85
    if success:
        print(f"{ship} refreshed market at {market}.")
    else:
        print(f"{ship} failed to refresh market at {market}.")
    fleet_resource_manager.set_ship_blocked_status(ship, False)
    return success    

In [39]:
await update_market("A", "XZY")

A is en-route to XZY.
A refreshed market at XZY.


True

In [5]:
system = "X1-QT34"
REFRESH_TIME = 60*30 # In seconds
CONTROLLER_ID = "PROBE-MANAGER-" + system
BASE_PRIO_MGR_PROBES = 100
REFRESH_PERIOD = 2 # Seconds between loops

In [44]:
q_all_markets = """
        with market_update_times as (
        select
            marketSymbol,
            min(ts_created) as ts_last_update,
            datetime(min(ts_created), 'unixepoch', 'localtime') as last_update
        from tradegoods_current
        group by marketSymbol
        )

        select
            distinct wp.symbol
        from 'nav.WAYPOINTS' wp

        inner join 'nav.TRAITS' t
        on wp.symbol = t.waypointSymbol
        and t.symbol = "MARKETPLACE"

        left join market_update_times mu
        on mu.marketSymbol = wp.symbol

        order by mu.ts_last_update asc
        """

q_exclude_fuel_only = f"""
        select
            distinct marketSymbol
        from tradegoods_current
        where symbol <> "FUEL"
        and ts_created < (strftime('%s', 'now') - {REFRESH_TIME})
        order by ts_created asc
"""

q_exclude_exchanges = f"""
        select
            distinct marketSymbol
        from tradegoods_current
        group by marketSymbol
        having sum(type = "IMPORT") > 0
           and sum(type = "EXPORT") > 0
           and ts_created < (strftime('%s', 'now') - {REFRESH_TIME})
        order by ts_created asc
"""

In [45]:
# Loop over these steps once ready
fleet = dict() #{s : {"task": None, "market": None} for s in fleet_resource_manager.get_controller_fleet(CONTROLLER_ID)}

In [46]:
# Find markets that have outdated info, sorted by most outdated and optionally filtered to exclude markets (all, exclude fuel-only markets, only import/export)
q = q_exclude_exchanges
targeted_markets = [s['market'] for s in fleet.values()]
market_order = [r[0] for r in io.read_list(q) if r[0] not in targeted_markets]
print(f"[INFO] {CONTROLLER_ID} is tracking {len(market_order)} markets.")

[INFO] PROBE-MANAGER-X1-QT34 is tracking 20 markets.


In [14]:
fleet_resource_manager.get_available_ships_in_systems(["X1-QT34"], "SATELLITE", 1, "PROBE-MANAGER-X1-QT34")

['RYVIOS-2', 'RYVIOS-3']

In [10]:
i = 1.2345
print(f"num = {i:.0f}")

num = 1


In [64]:
ships = fleet_resource_manager.get_controller_fleet("PROBE-MANAGER-X1-QT34")
for s in ships:
    print_nav_info(s)

RYVIOS-2 is DOCKED at X1-QT34-C40.
RYVIOS-3 is IN_ORBIT at X1-QT34-F53.


In [22]:
F_nav.get_ship_nav("RYVIOS-2")

{'departureTime': '2025-11-13T09:00:44.621Z',
 'waypointSymbol': 'X1-QT34-A3',
 'arrival': '2025-11-13T09:09:21.621Z',
 'status': 'IN_TRANSIT',
 'flightMode': 'DRIFT',
 'systemSymbol': 'X1-QT34',
 'symbol': 'RYVIOS-2'}

In [67]:
MI.get_import_export_markets_by_freshness('X1-QT34', 60*60*1)

['X1-QT34-F53',
 'X1-QT34-A3',
 'X1-QT34-F52',
 'X1-QT34-F51',
 'X1-QT34-H59',
 'X1-QT34-A4',
 'X1-QT34-E50',
 'X1-QT34-D42',
 'X1-QT34-K87',
 'X1-QT34-E45',
 'X1-QT34-D43',
 'X1-QT34-E46',
 'X1-QT34-E49',
 'X1-QT34-G55',
 'X1-QT34-G54',
 'X1-QT34-H56',
 'X1-QT34-E47',
 'X1-QT34-B7',
 'X1-QT34-J63',
 'X1-QT34-C40']

In [56]:
MI.find_closest_drones(fleet_resource_manager.get_controller_fleet('PROBE-MANAGER-X1-QT34'), 'X1-QT34-C40')

['RYVIOS-2', 'RYVIOS-3']

In [63]:
await scripts.update_market('RYVIOS-2', 'X1-QT34-C40')

True

In [66]:
fleet_resource_manager.release_fleet("PROBE-MANAGER-X1-QT34", force=True)

True